In [1]:
# import dependencies 
import tensorflow as tf
import numpy as np 
import pandas as pd 
from datetime import datetime, date, time
import os 
import glob
import csv

##importing dependencies 
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS 

token = "A1hvR7pmPINW66Wuxbb7dpwR74TukBJIC7IuP3ADhr4xTx0Zzx-JZxzgInUUFbikMsg_bNo2TI4QnbXnUIOkDg=="
org = "ThirdYearProject"
bucket = "a517e5ef0b9b8b98"

##connect to influxdb
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org,timeout=300000)

##DATA INTO INFLUXDB
write_api = client.write_api(write_options=SYNCHRONOUS,batch_size=10000)

file_list=pd.read_csv(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\dataInDB.csv")

In [6]:
#list = file_list.values.tolist()
#len(list)

#A1hvR7pmPINW66Wuxbb7dpwR74TukBJIC7IuP3ADhr4xTx0Zzx-JZxzgInUUFbikMsg_bNo2TI4QnbXnUIOkDg==
#de75cd94d8127ada
#synchronous specifies that there will be multiple batches of data being written at a time

934

In [2]:
list=[]

In [5]:
col_names=['HEADER']
csv_directory=r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\allCSV"
count=len(list)
for filename in os.listdir(csv_directory):
    if ([filename]) not in list:
        
        #keeping track of written csv files
        list.append([filename])
        count+=1
        
        #extracting tags for influxDB
        if filename.endswith(".csv"):
            tag_filename = filename.split("_segment_")[0]
            temp = filename.split("_segment_")[1]
            tag_segment = temp.split(".cs")[0]
            
        data = pd.read_csv(csv_directory+'/'+filename,index_col=None,names=col_names)
        
        #giving column titles to data
        data[['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']]=data['HEADER'].apply(lambda x: pd.Series(str(x).split("\t")))
        data['Channel'] = tag_filename
        data['Segment'] = tag_segment
        data.drop('HEADER', axis=1,inplace=True)
        data['Polarization Mode Dispersion'] = data['Polarization Mode Dispersion'].replace('?',0)
        convert_dict={'Q-Factor':float,
                     'Transmit Power (dBm)':float,
                     'Chromatic Dispersion(ps/nm)':float,
                      'Polarization Mode Dispersion':float
                     }
        data=data.astype(convert_dict)
        
        #clearning up data -> replacing ? with 0 (maybe should replace it with the average)
        #data['Polarization Mode Dispersion'] = pd.to_numeric(data['Polarization Mode Dispersion'], downcast="float",errors='coerce')
        #data.fillna(0)
        data['DateTime'] = pd.to_datetime(data['DateTime'], format="%Y.%m.%d.%H.%M.%S")
        data['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
        data.set_index(data['DateTime'], drop = True, inplace = True)

        #data.drop('DateTime', axis=1, inplace=True)
        data.drop('DateTime', axis=1, inplace=True)
    
        print(count)
        print(tag_filename)
        print(tag_segment)

        ##writing data into influxdb
        response = write_api.write(bucket, record=data, data_frame_measurement_name='Network Prediction', data_frame_tag_columns=['Channel','Segment'])

3
channel_1002
28
4
channel_1003
28
5
channel_1004
28
6
channel_1005
28
7
channel_1006
28
8
channel_1007
28
9
channel_1008
28
10
channel_1009
28
11
channel_100
3
12
channel_1010
28
13
channel_1011
28
14
channel_1012
28
15
channel_1013
28
16
channel_1014
28
17
channel_1015
28
18
channel_1016
28
19
channel_1017
28
20
channel_1018
28
21
channel_1019
28
22
channel_101
4
23
channel_1020
28
24
channel_1021
28
25
channel_1022
28
26
channel_1023
28
27
channel_1024
28
28
channel_1025
28
29
channel_1026
28
30
channel_1027
28
31
channel_1028
28
32
channel_1029
28
33
channel_102
4
34
channel_1030
28


ValueError: could not convert string to float: '?'

In [11]:
print(count)
print(len(list))
print(list)

4000
4000
[['channel_1001_segment_28.csv'], ['channel_1002_segment_28.csv'], ['channel_1003_segment_28.csv'], ['channel_1004_segment_28.csv'], ['channel_1005_segment_28.csv'], ['channel_1006_segment_28.csv'], ['channel_1007_segment_28.csv'], ['channel_1008_segment_28.csv'], ['channel_1009_segment_28.csv'], ['channel_100_segment_3.csv'], ['channel_1010_segment_28.csv'], ['channel_1011_segment_28.csv'], ['channel_1012_segment_28.csv'], ['channel_1013_segment_28.csv'], ['channel_1014_segment_28.csv'], ['channel_1015_segment_28.csv'], ['channel_1016_segment_28.csv'], ['channel_1017_segment_28.csv'], ['channel_1018_segment_28.csv'], ['channel_1019_segment_28.csv'], ['channel_101_segment_4.csv'], ['channel_1020_segment_28.csv'], ['channel_1021_segment_28.csv'], ['channel_1022_segment_28.csv'], ['channel_1023_segment_28.csv'], ['channel_1024_segment_28.csv'], ['channel_1025_segment_28.csv'], ['channel_1026_segment_28.csv'], ['channel_1027_segment_28.csv'], ['channel_1028_segment_28.csv'], ['c

In [12]:
#list.remove(['channel_1289_segment_34.csv'])
#print(len(list))

In [13]:
import csv
file_list = pd.DataFrame(list)
print(file_list)

                                0
0     channel_1001_segment_28.csv
1     channel_1002_segment_28.csv
2     channel_1003_segment_28.csv
3     channel_1004_segment_28.csv
4     channel_1005_segment_28.csv
...                           ...
3995   channel_997_segment_28.csv
3996   channel_998_segment_28.csv
3997   channel_999_segment_28.csv
3998     channel_99_segment_3.csv
3999      channel_9_segment_1.csv

[4000 rows x 1 columns]


In [16]:
filepath = r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\dataInDB.csv"
file_list.to_csv(filepath)

In [17]:
#rows = [[item] for item in file_list]
#print(rows)

file = open(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\dataInDB.csv", 'a+',newline ='')
with file:
    write = csv.writer(file)
    write.writerows(list)
    